This script creates short test videos from ALL complete gameplay videos. It requires a begining and an ending X_screen_total value from the event file.

WARNING: all complete gameplay videos must exist. If not, the script will fail.

In [ ]:
import os
import pandas as pd
from moviepy.editor import VideoFileClip

# Define base paths
base_path = '../data/raw_data/sub-01'
video_base_path = '../output/videos/videos_full'
output_video_base_path = '../output/videos/videos_cut'

# Import the combined CSV file
df = pd.read_csv('../data/datasets/combined_events_level-1_clean_outset_vid.csv')

# Ensure the output directory exists
if not os.path.exists(output_video_base_path):
    os.makedirs(output_video_base_path)

# Iterate through each unique combination of session, run, and repetition
for (subject, session, run, level, repetition) in df[['subject', 'session', 'run', 'level', 'repetition']].drop_duplicates().values:
    # Skip if any of the values are missing
    if pd.isna(subject) or pd.isna(session) or pd.isna(run) or pd.isna(level) or pd.isna(repetition):
        print(f"Skipping due to missing values: subject={subject}, session={session}, run={run}, level={level}, repetition={repetition}")
        continue
    
    # Define the video file name and path
    video_file_name = f'{subject}_{session}_task-shinobi_{run}_{level}_{repetition}.mp4'
    video_file_path = os.path.join(video_base_path, video_file_name)
    output_video_file_name = f'{subject}_{session}_task-shinobi_{run}_{level}_{repetition}_cut.mp4'
    output_video_file_path = os.path.join(output_video_base_path, output_video_file_name)

    # Check if the output video already exists
    if os.path.exists(output_video_file_path):
        print(f"Output video {output_video_file_name} already exists. Skipping.")
        continue
    
    # Load the event data for the current combination
    df_filtered = df[(df['subject'] == subject) &
                     (df['session'] == session) &
                     (df['run'] == run) &
                     (df['level'] == level) &
                     (df['repetition'] == repetition)]

    # Load the video file with moviepy
    video = VideoFileClip(video_file_path)

    # Define the start and end x_screen_total values
    start_x_screen_total = 4550  # <-- Change this value to adjust the START of the video
    end_x_screen_total = 5150    # <-- Change this value to adjust the END of the video

    # Get start and end onsets from the dataframe
    start_onset = float(df_filtered[(df_filtered['X_screen_total'] >= start_x_screen_total)]['onset_vid'].values[0])
    end_onset = float(df_filtered[(df_filtered['X_screen_total'] >= end_x_screen_total)]['onset_vid'].values[0])

    # Cut the video using the onset times
    video_cut = video.subclip(start_onset, end_onset)

    # Print video information
    fps = video.fps
    n_frames = int(video_cut.reader.nframes)
    print(f'Video start time (s): {start_onset}')
    print(f'Video end time (s): {end_onset}')
    print(f'Total time (s): {n_frames / fps:.3f}')
    print(f"Video frames per second (fps): {fps}")
    print(f'Total frame number: {n_frames}')
    
    # Write the output video file with audio
    video_cut.write_videofile(output_video_file_path, codec='libx264', audio_codec='aac', fps=fps)
    print('-----------------------------------')